In [1]:
import syft as sy
import numpy as np
import pandas as pd
from syft.core.adp.entity_list import EntityList
from time import time

In [3]:
t0 = time()
df = pd.read_parquet("/home/ruchi/1B_rows_dataset_sample.parquet")
# df = pd.read_parquet("../Ishan/1M_rows_dataset_sample.parquet")
tf = time() - t0

print(f"Time taken to read file with {df.shape[0]/1e6} million rows : {tf} seconds")

Time taken to read file with 1000.0 million rows : 93.70291066169739 seconds


In [4]:
scale = 10_000  # This is put here to stop int32 overflow errors


t0 = time()
impressions = df['impressions'].to_numpy(np.int64) // scale
data_subjects = EntityList.from_series(df['user_id'])
tf = time() - t0

print(f"Time taken to create inputs for Syft Tensor: {tf} seconds")

Time taken to create inputs for Syft Tensor: 51.09363269805908 seconds


In [9]:
t0 = time()
tweets_data = sy.Tensor(impressions).private(min_val=750_000/scale, max_val=20e6/scale, entities = data_subjects,ndept=True)  # RUn this for 1 billion rows
# tweets_data = sy.Tensor(impressions).private(min_val=500/scale, max_val=25000/scale, entities = data_subjects,ndept=True)  # Run this for 1 million rows
tf = time() - t0

print(f"Time make Private Syft Tensor: {tf} seconds")

Time make Private Syft Tensor: 2.2217493057250977 seconds


In [6]:
domain_node = sy.login(email="info@openmined.org",password="changethis",port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into hardcore_gardner... done!


In [10]:
%%time
name = f"Tweets data- {time()}"

domain_node.load_dataset(
    assets={name: tweets_data},
    name=name,
    description=" Tweets- 1B rows",
    use_blob_storage=True
)

Loading dataset... checking asset types...                                                                                                                                    

Uploading Tweets data- 1647986234.3514206: 100%|██| 7/7 [00:08<00:00,  1.22s/it]


Loading dataset... uploading... SUCCESS!                        

Run <your client variable>.datasets to see your new dataset loaded into your machine!
CPU times: user 39.4 s, sys: 10.1 s, total: 49.5 s
Wall time: 1min 6s


In [11]:
domain_node.datasets

Idx,Name,Description,Assets,Id
[0],Tweets data- 1647986108.1445184,Tweets- 1B rows,"[""Tweets data- 1647986108.1445184""] ->",e2b07225-b6ee-4b07-957a-318bf9a1ec99
[1],Tweets data- 1647986234.3514206,Tweets- 1B rows,"[""Tweets data- 1647986234.3514206""] ->",104b18b2-01b4-42c8-9162-65045aee09c2


In [12]:
data = domain_node.datasets[-1][name]

In [13]:
%%time
sum_result = data.sum()
sum_result.block

CPU times: user 3.05 s, sys: 238 ms, total: 3.28 s
Wall time: 1min 46s


<TensorPointer -> hardcore_gardner:a9b9a3d295b745a685790b79e85f3a13>

In [14]:
sum_result.exists

True

In [15]:
published_result = sum_result.publish(sigma=1e10)

In [16]:
published_result.exists

True

In [17]:
published_result

<FloatPointer -> hardcore_gardner:14669bffb0d34f63a93322fab4ef416f>

In [21]:
published_result.request()

In [22]:
domain_node.requests

,Name,Email,Role,Request Type,Status,Reason,Request ID,Requested Object's ID,Requested Object's tags,Requested Budget,Current Budget
0,Jane Doe,info@openmined.org,Owner,DATA,pending,,<UID: 2352122b71e24e6c9826501d7eb3cd28>,<UID: 14669bffb0d34f63a93322fab4ef416f>,[],None,None


In [23]:
domain_node.requests[-1].accept()

In [24]:
domain_node.requests

,Name,Email,Role,Request Type,Status,Reason,Request ID,Requested Object's ID,Requested Object's tags,Requested Budget,Current Budget
0,Jane Doe,info@openmined.org,Owner,DATA,accepted,,<UID: 2352122b71e24e6c9826501d7eb3cd28>,<UID: 14669bffb0d34f63a93322fab4ef416f>,[],None,None


In [25]:
result_of_our_hard_labour = published_result.get()

# WE GOT AN OVERFLOW ERROR
- doesn't occur with 1M rows
- publish had completed, tho it took 602 seconds

In [26]:
result_of_our_hard_labour

array(-2147483648, dtype=int32)

In [22]:
impressions.sum()

11941995258

In [23]:
def percentage_error(true_value, noisy_value):
    return (true_value - noisy_value)/true_value * 100

In [24]:
percentage_error(true_value=impressions.sum(), noisy_value=result_of_our_hard_labour)

96.35989431741909

In [13]:
(1644527104 - 11941995258)/11941995258 * 100

-86.22904239642598

In [14]:
11941995258 /1e6

11941.995258